In [1]:
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

c:\users\shitij\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
def get_data():
    movie_data = pd.read_csv('movie_data.csv')
    movie_data['original_title'] = movie_data['original_title'].str.lower()
    return movie_data

In [3]:
def combine_data(data):
        data_recommend = data.drop(columns=['movie_id', 'original_title','plot'])
        data_recommend['combine'] = data_recommend[data_recommend.columns[0:2]].apply(
                                                                        lambda x: ','.join(x.dropna().astype(str)),axis=1)
        
        data_recommend = data_recommend.drop(columns=[ 'cast','genres'])
        return data_recommend

In [4]:
def transform_data(data_combine, data_plot):
        count = CountVectorizer(stop_words='english')
        count_matrix = count.fit_transform(data_combine['combine'])

        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(data_plot['plot'])

        combine_sparse = sp.hstack([count_matrix, tfidf_matrix], format='csr')
        cosine_sim = cosine_similarity(combine_sparse, combine_sparse)
        
        return cosine_sim

In [5]:
def recommend_movies(title, data, combine, transform):
        indices = pd.Series(data.index, index = data['original_title'])
        index = indices[title]

        sim_scores = list(enumerate(transform[index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:21]
        
        movie_indices = [i[0] for i in sim_scores]

        movie_id = data['movie_id'].iloc[movie_indices]
        movie_title = data['original_title'].iloc[movie_indices]
        movie_genres = data['genres'].iloc[movie_indices]

        recommendation_data = pd.DataFrame(columns=['Movie_Id','Name', 'Genres'])

        recommendation_data['Movie_Id'] = movie_id
        recommendation_data['Name'] = movie_title
        recommendation_data['Genres'] = movie_genres

        return recommendation_data

In [6]:
def results(movie_name):
        movie_name = movie_name.lower()

        find_movie = get_data()
        combine_result = combine_data(find_movie)
        transform_result = transform_data(combine_result,find_movie)

        if movie_name not in find_movie['original_title'].unique():
                return 'Movie not in Database'

        else:
                recommendations = recommend_movies(movie_name, find_movie, combine_result, transform_result)
                return recommendations.to_dict('records')


In [7]:
results('The Dark Knight')

[{'Movie_Id': '49026',
  'Name': 'the dark knight rises',
  'Genres': "['Action', 'Crime', 'Drama']"},
 {'Movie_Id': '272',
  'Name': 'batman begins',
  'Genres': "['Action', 'Crime', 'Drama']"},
 {'Movie_Id': '378237',
  'Name': "amidst the devil's wings",
  'Genres': "['Drama', 'Action', 'Crime']"},
 {'Movie_Id': '168672',
  'Name': 'american hustle',
  'Genres': "['Drama', 'Crime']"},
 {'Movie_Id': '7873',
  'Name': 'harsh times',
  'Genres': "['Crime', 'Drama', 'Thriller']"},
 {'Movie_Id': '164457',
  'Name': 'out of the furnace',
  'Genres': "['Thriller', 'Drama', 'Crime']"},
 {'Movie_Id': '5955',
  'Name': 'the pledge',
  'Genres': "['Crime', 'Drama', 'Mystery']"},
 {'Movie_Id': '11322',
  'Name': 'public enemies',
  'Genres': "['History', 'Crime', 'Drama']"},
 {'Movie_Id': '147441',
  'Name': 'exodus: gods and kings',
  'Genres': "['Adventure', 'Drama', 'Action']"},
 {'Movie_Id': '267860',
  'Name': 'london has fallen',
  'Genres': "['Action', 'Crime', 'Thriller']"},
 {'Movie_Id